In [9]:
import pandas as pd
import numpy as np
agency = pd.read_csv('../datasets/asana/gtfs_data/agency.txt')
routes = pd.read_csv('../datasets/asana/gtfs_data/routes_utf8.txt', sep="\t")
stop_times = pd.read_csv('../datasets/asana/gtfs_data/stop_times.txt', sep="\t")
stops = pd.read_csv('../datasets/asana/gtfs_data/stops.txt', sep="\t")
trips = pd.read_csv('../datasets/asana/gtfs_data/trips.txt', sep="\t")



In [10]:
modeling_structure_df = pd.DataFrame()

route_travel_time = pd.DataFrame()
arrival_time = pd.DataFrame()
delay_prediction = pd.DataFrame()
# most descriptive columns from the gtfs format for modeling that i currently identified are:
#  from stop_times : stop_id, arrival_time, departure_time
#  from stops : stop_id, stop_lat, stop_lon
#  from trips : shape_id, route_id (To relate the info, basically a join table)

# The features that i imagine i could extract to get more info are:

# average_stop_times (Would describe an average of time from stop to stop per route)
# a more defined and specific attributes below:

#avg_seconds_between_stops
#std_seconds_between_stops

# hour_of_day (Pretty descriptive name)
# stop_quantity (A sum of the bus stops)
# stop_density_500m (This measures how conglomerated a zone is in the 500m radious)


# gps_data = pd.read_csv('../datasets/asana/gps_data.csv') # thing is so big i need to put attributes here because checking takes long - id,deviceid,devicetime,latitude,longitude,speed,route_direction,route_id,route_guid,device_guid

# Fordward here im going to explain the structure of how this dataset is going to be analized

# I'm going to add steps to it: 
# 1st Step is going to ananlyze just structural, coordinates, stop density, etc. So i can see for example if using a delay in time target (Which will probably be the most used target in this notebook), i can see how much the data is giving in comparison 
# ------- (Here on step 1, will be using normal train_test_split and will be trained with diferent models and compared with CV)
# to 2nd Step which will only use context data such as speeds, times, etc. || i will search for most significant/descriptive attributes for each, and look into compare important stadistic values such as R^2 and others
# ------- (Here on step 2, will be using time_series_split with lags, so the model also finds relationship in between the difference in time)
# Finally on step 3, depending on which attributes tend to be the most meaningful and tune the best models to find a in between in overfitting and underfitting 


# Now onto other other targets its going to be the effective_speeds
# Using the GTFS-RT, finding the actual time taken to complete a route, we can find the delays in the actual schedulet time, which will be useful to create a "isDelayed" target using a specific treshold
# Clustering could be applied using the times, the route and locations and stop densities to find congestion zones



In [11]:
# First row im going to compute is the trip length in KM with sets of longitudes using the Haversine formula |||| reference : https://www.youtube.com/watch?v=cD6pUr_5RtE

In [12]:
import math

def distance_between(lat1, lon1, lat2, lon2):
    earthRadius = 6371
    dlat = degreesToRadians(lat2-lat1)
    dlon = degreesToRadians(lon2-lon1) 
    rad = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(degreesToRadians(lat1)) * math.cos(degreesToRadians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    rad = 2 * math.atan2(math.sqrt(rad), math.sqrt(1-rad))
    distance = earthRadius * rad
    return distance
    
def degreesToRadians(deg):
    return deg * (math.pi / 180) 

In [13]:
#stops.head()


In [14]:
trips.shape

(19769, 7)

In [15]:
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,10406,07:20:44,07:20:44,5d11962d-6412-4c7c-984d-f4ee5eb63859,41
1,27104,20:14:37,20:14:57,5d11962d-6412-4c7c-984d-f4ee5eb63859,40
2,25687,17:45:24,17:45:43,5d11962d-6412-4c7c-984d-f4ee5eb63859,41
3,25541,10:42:59,10:42:59,5d11962d-6412-4c7c-984d-f4ee5eb63859,42
4,22444,13:40:10,13:41:00,5d11962d-6412-4c7c-984d-f4ee5eb63859,35


In [16]:
#routes.head()

In [17]:
sorted_stop_times = stop_times.sort_values(by=['trip_id', 'stop_sequence'], ascending=[True, True])


In [18]:
sorted_route_stops_coords = sorted_stop_times.merge(stops[["stop_id", "stop_lat", "stop_lon"]], on="stop_id", how="left")


In [19]:
sorted_route_stops_coords["prev_lat"] = sorted_route_stops_coords["stop_lat"].shift(1)
sorted_route_stops_coords["prev_lon"] = sorted_route_stops_coords["stop_lon"].shift(1)


In [20]:
sorted_route_stops_coords["segment_distance"] = sorted_route_stops_coords.apply(
    lambda row: distance_between(
        row["prev_lat"],
        row["prev_lon"],
        row["stop_lat"],
        row["stop_lon"]
    ) if pd.notnull(row["prev_lat"]) else 0,
    axis=1
)


In [21]:
route_distance_df = (
    sorted_route_stops_coords.groupby("trip_id")["segment_distance"]
      .sum()
      .reset_index(name="total_distance_km")
)


In [22]:
modeling_structure_df = trips

In [23]:

modeling_structure_df = modeling_structure_df.drop(columns=['service_id','vehicle_id', 'direction_id'])

In [24]:
modeling_structure_df = modeling_structure_df.merge(route_distance_df, on="trip_id", how="left")


In [25]:
stop_count = (
    stop_times.groupby("trip_id")['stop_sequence'].count()
    .reset_index(name="total_stops")
)

In [26]:
modeling_structure_df = modeling_structure_df.merge(stop_count, on="trip_id", how="left")


In [27]:
def calcEffectiveSpeed(start_time_s, end_time_s, total_distance_km):
    if end_time_s < start_time_s:  #this means it cycled to next day (which i find improbable, but still)
        end_time_s+=86400
    duration = end_time_s - start_time_s

    if duration <= 0:
        return np.nan #invalid data
    
    return total_distance_km * 3600/duration
    

In [28]:
modeling_structure_df["start_sec"] = pd.to_timedelta(modeling_structure_df["start_time"]).dt.total_seconds()
modeling_structure_df["end_sec"]   = pd.to_timedelta(modeling_structure_df["end_time"]).dt.total_seconds()
modeling_structure_df = modeling_structure_df.drop(columns=['start_time', 'end_time']) 

modeling_structure_df["effective_scheduled_speed_kmh"] = modeling_structure_df.apply(
    lambda row: calcEffectiveSpeed(
        row["start_sec"],
        row["end_sec"],
        row["total_distance_km"],
    ),
    axis=1
)


In [29]:
modeling_structure_df

,route_id,trip_id,total_distance_km,total_stops,start_sec,end_sec,effective_scheduled_speed_kmh
0,d626b854-27aa-41d4-8625-ebafa73d8f21,23809,37.516956,36,38521.0,44990.0,20.878195
1,d626b854-27aa-41d4-8625-ebafa73d8f21,23705,37.516956,36,28733.0,33312.0,29.495751
2,d626b854-27aa-41d4-8625-ebafa73d8f21,23547,21.245568,39,70904.0,75385.0,17.068522
3,d626b854-27aa-41d4-8625-ebafa73d8f21,23497,21.077345,36,62522.0,70901.0,9.055787
4,d626b854-27aa-41d4-8625-ebafa73d8f21,23455,21.353914,40,57835.0,62517.0,16.419071
...,...,...,...,...,...,...,...
19764,c453217f-9f9d-49ce-8b32-14a6b7013691,26011,38.988103,44,53273.0,58278.0,28.043391
19765,c453217f-9f9d-49ce-8b32-14a6b7013691,25978,20.239318,38,48387.0,53259.0,14.955161
19766,c453217f-9f9d-49ce-8b32-14a6b7013691,25951,25.623639,44,43346.0,48379.0,18.328055
19767,c453217f-9f9d-49ce-8b32-14a6b7013691,25902,36.667677,39,36459.0,43292.0,19.318548


In [30]:
routes_per_stop_df = (stop_times.groupby("stop_id")["trip_id"].count().reset_index(name="routes_per_stop"))
routes_per_stop_df["routes_per_stop"] = routes_per_stop_df["routes_per_stop"] - 1 #exclude current route 


merged = stop_times.merge(routes_per_stop_df, on="stop_id", how="left")

In [31]:
overlap_score = (
    merged.groupby('trip_id')["routes_per_stop"]
    .mean()
    .reset_index(name="overlap_score")
)

In [32]:
modeling_structure_df = modeling_structure_df.merge(overlap_score, on="trip_id", how="left")

In [33]:
modeling_structure_df["stop_density"] = (modeling_structure_df["total_stops"]-1) / modeling_structure_df["total_distance_km"] #substracting 1 to stops to measure the actual segments between stops

In [34]:
modeling_structure_df = modeling_structure_df.drop(columns=["total_stops", "start_sec", "end_sec"])

In [35]:
modeling_structure_df 

,route_id,trip_id,total_distance_km,effective_scheduled_speed_kmh,overlap_score,stop_density
0,d626b854-27aa-41d4-8625-ebafa73d8f21,23809,37.516956,20.878195,4220.777778,0.932911
1,d626b854-27aa-41d4-8625-ebafa73d8f21,23705,37.516956,29.495751,4220.777778,0.932911
2,d626b854-27aa-41d4-8625-ebafa73d8f21,23547,21.245568,17.068522,3834.871795,1.788608
3,d626b854-27aa-41d4-8625-ebafa73d8f21,23497,21.077345,9.055787,4220.777778,1.660551
4,d626b854-27aa-41d4-8625-ebafa73d8f21,23455,21.353914,16.419071,3873.925000,1.826363
...,...,...,...,...,...,...
19764,c453217f-9f9d-49ce-8b32-14a6b7013691,26011,38.988103,28.043391,4809.590909,1.102901
19765,c453217f-9f9d-49ce-8b32-14a6b7013691,25978,20.239318,14.955161,5183.394737,1.828125
19766,c453217f-9f9d-49ce-8b32-14a6b7013691,25951,25.623639,18.328055,4809.590909,1.678138
19767,c453217f-9f9d-49ce-8b32-14a6b7013691,25902,36.667677,19.318548,5237.820513,1.036335


In [36]:
modeling_structure_df[["total_distance_km", "stop_density", "overlap_score", "effective_scheduled_speed_kmh"]].corr() # stop density and total distance km are very correlated, but will keep it because 
                                                                                                                      #i will probably just use gradient boosting and it already handles it


,total_distance_km,stop_density,overlap_score,effective_scheduled_speed_kmh
total_distance_km,1.000000,-0.934410,-0.062129,0.700862
stop_density,-0.934410,1.000000,0.141396,-0.632685
overlap_score,-0.062129,0.141396,1.000000,-0.216111
effective_scheduled_speed_kmh,0.700862,-0.632685,-0.216111,1.000000


In [37]:
modeling_structure_df = modeling_structure_df.drop(columns=["route_id", "trip_id"])

In [38]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score, KFold

X = modeling_structure_df.drop(columns=["effective_scheduled_speed_kmh"])
y = modeling_structure_df["effective_scheduled_speed_kmh"]


In [39]:
linearRegression_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("model", LinearRegression())
])

In [40]:
xgb_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("model", XGBRegressor(
    n_estimators=200,
    learning_rate=0.3,
    max_depth=6,
    ))
])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2
)
# First training with just structural attributes

xgb_scores = cross_val_score(xgb_pipeline, X_train, y_train, cv=5)
xgb_scores_mae = cross_val_score(xgb_pipeline, X_train, y_train, cv=5,scoring="neg_mean_absolute_error")
xgb_scores_rmae = cross_val_score(xgb_pipeline, X_train, y_train, cv=5,scoring='neg_mean_squared_error')
LR_scores = cross_val_score(linearRegression_pipeline, X_train, y_train, cv=5)
LR_scores_mae = cross_val_score(linearRegression_pipeline, X_train, y_train, cv=5,scoring="neg_mean_absolute_error")
LR_scores_rmae = cross_val_score(linearRegression_pipeline, X_train, y_train, cv=5,scoring='neg_mean_squared_error')

In [41]:
print(f"xgb R^2: {xgb_scores.mean()}, MAE: {xgb_scores_mae.mean()} RMAE: {xgb_scores_rmae.mean()}")
print(f"LR R^2: {LR_scores.mean()}, MAE: {LR_scores_mae.mean()}RMAE: {LR_scores_rmae.mean()}")


xgb R^2: 0.5637931264840287, MAE: -2.8076337450308775 RMAE: -15.644590105167094
LR R^2: 0.529245982574855, MAE: -3.0587741112747207RMAE: -16.892715443364594


In [42]:
# now feature engineering with time data GTFS-RT data: 
# Columns : id,deviceid,devicetime,latitude,longitude,speed,route_direction,route_id,route_guid,device_guid

In [43]:
# here will use duck db because making operations on a 115 Million ish table sucks with low ram

In [44]:
import duckdb

con = duckdb.connect()

con.execute("""
    CREATE TABLE gps_sorted AS
    SELECT *
    FROM 'gps_clean.parquet'
    ORDER BY deviceid, devicetime
""")



In [45]:
con.execute("""
    CREATE TABLE segments AS
    SELECT * FROM read_csv_auto('../datasets/asana/segment_level_data.csv');
""")

In [46]:
con.execute("DESCRIBE gps_sorted").fetchdf()


,column_name,column_type,null,key,default,extra
0,id,BIGINT,YES,None,None,None
1,deviceid,BIGINT,YES,None,None,None
2,devicetime,TIMESTAMP,YES,None,None,None
3,latitude,DOUBLE,YES,None,None,None
4,longitude,DOUBLE,YES,None,None,None
5,speed,DOUBLE,YES,None,None,None
6,route_direction,VARCHAR,YES,None,None,None
7,route_id,BIGINT,YES,None,None,None
8,route_guid,VARCHAR,YES,None,None,None
9,device_guid,VARCHAR,YES,None,None,None


In [54]:
con.execute("""
    COPY (
        SELECT *
        FROM read_csv_auto('../datasets/asana/segment_level_data.csv')
    )
    TO 'segments.parquet'
    (FORMAT PARQUET);
""")

In [48]:
con.execute("DESCRIBE segments").fetchdf()


,column_name,column_type,null,key,default,extra
0,date,DATE,YES,None,None,None
1,deviceid,BIGINT,YES,None,None,None
2,direction,BIGINT,YES,None,None,None
3,segment,BIGINT,YES,None,None,None
4,start_point,BIGINT,YES,None,None,None
5,end_point,BIGINT,YES,None,None,None
6,start_time,TIMESTAMP,YES,None,None,None
7,run_time_in_seconds,BIGINT,YES,None,None,None
8,dwell_time_in_seconds,BIGINT,YES,None,None,None
9,arrival_time,TIMESTAMP,YES,None,None,None


In [49]:
con.execute("DESCRIBE segments").fetchdf()


,column_name,column_type,null,key,default,extra
0,date,DATE,YES,None,None,None
1,deviceid,BIGINT,YES,None,None,None
2,direction,BIGINT,YES,None,None,None
3,segment,BIGINT,YES,None,None,None
4,start_point,BIGINT,YES,None,None,None
5,end_point,BIGINT,YES,None,None,None
6,start_time,TIMESTAMP,YES,None,None,None
7,run_time_in_seconds,BIGINT,YES,None,None,None
8,dwell_time_in_seconds,BIGINT,YES,None,None,None
9,arrival_time,TIMESTAMP,YES,None,None,None


In [72]:
segments_from_trip = con.execute("""
    SELECT
        AVG(g.speed) AS avg_speed,
        VAR_SAMP(g.speed) AS variance,
        EXTRACT(EPOCH FROM MAX(g.devicetime) - MIN(g.devicetime)) AS real_duration,
        s.trip_id
        
    FROM gps_sorted g
    JOIN segments s
      ON g.deviceid = s.deviceid
     AND g.devicetime BETWEEN s.start_time AND s.arrival_time
    GROUP BY
        s.trip_id,
""").fetchdf()


In [86]:
con.execute("""
    CREATE TABLE trip_segments_full AS
    SELECT * FROM segments_from_trip
""")

CatalogException: Catalog Error: Table with name "trip_segments_full" already exists!

In [55]:
con.execute("""
    COPY (
        SELECT *
        FROM read_csv_auto('../datasets/asana/gtfs_data/trips.txt')
    )
    TO 'trips.parquet'
    (FORMAT PARQUET);
""")

In [57]:
con.execute("""
    CREATE TABLE trips AS
    SELECT * FROM read_parquet('trips.parquet')
""")

In [67]:
con.execute("""
COPY (SELECT *, EXTRACT(EPOCH FROM end_time) - EXTRACT(EPOCH FROM start_time) AS scheduled_duration FROM trips) TO 'trip_tweak.parquet' (FORMAT PARQUET);
""")


In [77]:
df = con.execute("SELECT * FROM trip_segments_full").fetchdf()


In [84]:
df

,deviceid,start_time,arrival_time,avg_speed,variance,real_duration,trip_id
0,779,2024-08-01 06:46:24,2024-08-01 06:47:24,23.754782,99.055063,60.0,1475
1,779,2024-08-01 07:16:13,2024-08-01 07:16:19,6.317221,15.255114,6.0,1475
2,779,2024-08-01 07:02:21,2024-08-01 07:03:22,26.416029,49.194995,61.0,1475
3,779,2024-08-01 07:34:31,2024-08-01 07:35:06,33.912220,57.521103,35.0,1475
4,779,2024-08-01 09:34:43,2024-08-01 09:35:23,15.948839,87.791062,40.0,1565
...,...,...,...,...,...,...,...
785969,4909,2024-09-11 16:26:30,2024-09-11 16:28:51,10.895219,48.147388,141.0,26030
785970,4909,2024-09-11 17:21:59,2024-09-11 17:22:35,21.839679,18.647080,36.0,26030
785971,4909,2024-09-11 17:49:28,2024-09-11 17:54:17,8.014105,107.670339,289.0,26074
785972,4909,2024-09-11 18:50:35,2024-09-11 18:52:27,9.583976,28.016389,112.0,26074


In [87]:
con.execute("""CREATE OR REPLACE TABLE trip_segments_full AS 
SELECT
    trip_id,
    ANY_VALUE(deviceid) AS deviceid,
    SUM(real_duration) AS trip_real_duration,
    SUM(avg_speed * real_duration) / SUM(real_duration) AS trip_avg_speed
FROM trip_segments_full
GROUP BY trip_id""");


In [88]:
df = con.execute("SELECT * FROM trip_segments_full").fetchdf()


In [89]:
df

,trip_id,deviceid,trip_real_duration,trip_avg_speed
0,26913,999,3131.0,19.299749
1,27000,999,3453.0,17.290785
2,27096,999,3489.0,17.442873
3,2262,1019,5956.0,18.054863
4,7470,1019,4060.0,18.115091
...,...,...,...,...
19764,16867,794,3510.0,19.648376
19765,24843,1039,4101.0,17.483899
19766,758,1118,4112.0,16.628798
19767,10422,1145,6103.0,16.147548


In [ ]:
con.execute("""""")

In [93]:
segments_from_trip = con.execute(""" 
    SELECT 
        s.trip_id,
        s.trip_real_duration,
        s.trip_avg_speed,
        s.deviceid,
        (s.trip_real_duration - t.scheduled_duration) AS delay

    FROM trip_segments_full s
    JOIN trip_tweak.parquet t
      ON s.trip_id = t.trip_id
""").fetchdf()

In [94]:
segments_from_trip

,trip_id,trip_real_duration,trip_avg_speed,deviceid,delay
0,26913,3131.0,19.299749,999,-1245.0
1,27000,3453.0,17.290785,999,-1482.0
2,27096,3489.0,17.442873,999,-1282.0
3,2262,5956.0,18.054863,1019,-1246.0
4,7470,4060.0,18.115091,1019,-975.0
...,...,...,...,...,...
19764,16867,3510.0,19.648376,794,-1158.0
19765,24843,4101.0,17.483899,1039,-4265.0
19766,758,4112.0,16.628798,1118,-1191.0
19767,10422,6103.0,16.147548,1145,-4267.0


In [95]:
trip_tweak = con.execute("""SELECT * FROM trip_tweak.parquet""").fetchdf()

In [97]:
trip_tweak.iloc[0]

route_id              d626b854-27aa-41d4-8625-ebafa73d8f21
service_id                              service_2024-09-20
trip_id                                              23809
direction_id                                             1
start_time                                        10:42:01
end_time                                          12:29:50
vehicle_id            07f4d1a4-b216-4906-a4b7-5a3d7af38266
scheduled_duration                                  6469.0
Name: 0, dtype: object

In [98]:
segments_from_trip.iloc[0]

trip_id               26913.000000
trip_real_duration     3131.000000
trip_avg_speed           19.299749
deviceid                999.000000
delay                 -1245.000000
Name: 0, dtype: float64